In [22]:
# Import the required libraries for webscraping
from bs4 import BeautifulSoup
import requests
import re
# Import requried utility libraries
import os

### Get urls from the input folder.
It is assumed that the input folder is at the same level with src, and the file holding the urls is called from_urls. Also it is assumed that the urls are stored and separated by newlines in the input file.\
It is also assumed that only the first two lines of the input file will be valid URLs. No verificiation is carried out for the input data as it is outside the assignment scope


In [23]:
# Initialize the url list
url_dict = {
    'season_126': '',
    'season_womens_seven': ''
}

# Define the import file path
INPUT_FILE_NAME = 'from_urls'

url_file_fn = os.path.join(
    os.path.abspath('..'), 
    'input',
    INPUT_FILE_NAME
)

# Open the file and read the urls into the url list
url_file = open(url_file_fn, 'r')
for k, v in url_dict.items():
    line = url_file.readline()
    if line:
        url_dict[k] = line.strip()
    else:
        pass
url_file.close()

### Request data from Wikipedia for AFL Season 126 and AFL Women's Season Seven
The raw data will also be saved into a dictionary of two BeautifulSoup objects

In [ ]:
# Initialize the BeautifulSoup object dictionary
bs_dict = {
    'season_126': None,
    'season_womens_seven': None
}

# Iterate through url_dict and save the BeautifulSoup object to bs_dict
for k, v in url_dict.items():
    response = requests.get(v)
    html = response.text
    bs_dict[k] = BeautifulSoup(html, 'html.parser')
    response.close()


### Extract from the BeautifuleSoup projects for the tables under "Home-and-away season"

In [ ]:
# TO-DO helper functoin to extract the required elements of the round tables
def extract_round_and_match_elements(soup_obj):
    # Note: round elements should be a match bs object list
    result_round_elements = []
    found_round_list = soup_obj.find_all('to-do')
    for round in found_round_list:
        round_number = round.find('to-do')
        matches = round.find_all('to-do')
        for match in matches:
            game_datetime = match.find('to-do')
            home = match.find('to-do')
            result = match.find('to-do')
            away = match.find('to-do')
            location = match.find('to-do')
            result_round_elements.append(
                {
                    'round_number': round_number.text,
                    'datetime': game_datetime.text,
                    'home': home.text,
                    'result': result.text,
                    'away': away.text,
                    'location': location.text
                }
            )
    return result_round_elements

In [19]:
# TO-DO: Iterate through the BeautifulSoup object and extract all the ROUND tables under 
# body -> div class mw-page-container -> div class mw-content-container
# -> div class vector-body ve-init-mw-desktopArticleTarget-targetContainer
# -> div class mw-body-content mw-content-ltr
# -> div class mw-parser-output -> ALL non-"wikitable" class tables

# Initialize a match_dict to store the extracted match element lists
round_dict = {
    'season_126': [],
    'season_womens_seven': []
}
for k, v in bs_dict.items():
    round_dict[k] = extract_round_and_match_elements(v)


### Extract the match information from the round_dict and save to a dictionary of matches

In [ ]:
# TO-DO: Iterate through the round elements and save the match elements to a list of strings
def extract_match_elements_to_strings(rounds):
    match_string_list = []
    re_pattern_dict = {
        'datetime': 'some regex pattern',
        'home': 'some regex pattern',
        'result': 'some regex pattern',
        'away': 'some regex pattern',
        'location': 'some regex pattern'
    }
    matched_re_group_dict = {
        'datetime': None,
        'home': None,
        'result': None,
        'away': None,
        'location': None
    }
    for match in rounds:
        # Hint: use regex here to find the below values from a round element
        for k, v in re_pattern_dict.items():
            matched_re_group_dict[k] = re.match(v, match[k])
        this_match_str = ','.join([
            match['round_number'],
            matched_re_group_dict['datetime'].group(0),
            matched_re_group_dict['home'].group(0)
            # to-do: add all values required
        ])
        match_string_list.append(this_match_str)
    return match_string_list

In [20]:
# TO-DO: iterate through the round_dict and extract the following information for each match
# RegEx is used here per example 3.5 Part 3
"""
Round Number (example: "1")
Day of the game (example: "Friday")
Date of the game (example: "19-Aug")
Time of the game (example: "7:50 pm")
First team name (example: "Brisbane Lions")
First team score in points only (example: "57")
First team win status (either "def. by" or "def.")
Second team name (example: "Melbourne")
Second team score in points only (example: "115")
Game location (example: "The Gabba")
Stadium Attendees (example: "32172")
"""

# Initialize a match_dict to store the extracted match element lists
csv_title_row = ','.join([
    'Round Number',
    'Day of the game',
    'Date of the game',
    'Time of the game',
    'First team name',
    'First team score',
    'First team win status',
    'Second team name',
    'Second team score',
    'Game location',
    'Stadium Attendees'
])
match_string_list = []
for k, v in round_dict.items():
    match_string_list.extend(extract_match_elements_to_strings(v))

### Save the result to a csv output
It is assumed that the output folder is at the same level as the src folder \
It is also assumed that a csv file output named scraped_match_table.csv file will exist and contain the result \
The file will be overwritten regardless of its existing contents and no verification of existence will be carried out\
since it is outside the assignment scope

In [21]:
# Define the output file path
OUTPUT_FILE_NAME = 'scraped_match_table.csv'
scraped_match_table_fn = os.path.join(
    os.path.abspath('..'), 
    'output',
    OUTPUT_FILE_NAME
)

# Write the result_list to the output file
result_file = open(scraped_match_table_fn, 'w')
for result_str in match_string_list:
    result_file.write(f"{result_str}\n")
result_file.close()